In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from numpy import expand_dims
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot
from matplotlib.patches import Rectangle

2022-01-28 09:40:51.784103: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-28 09:40:51.784148: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [4]:
# make suare images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape: ", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape:  (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
# load dense softmax model
model = load_model('./Data/conv_flatten_softmax_model.h5')

2022-01-28 09:41:09.245894: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-28 09:41:09.245943: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-28 09:41:09.245982: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu): /proc/driver/nvidia/version does not exist
2022-01-28 09:41:09.246422: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
a_val_img = x_test[0]

In [8]:
# Cool
model.predict(np.array([a_val_img]))

array([[2.7691417e-19, 2.1103689e-18, 6.9270913e-15, 4.4732502e-08,
        1.5199857e-23, 2.5408093e-17, 8.9121865e-38, 1.0000000e+00,
        1.3648736e-15, 1.2447636e-13]], dtype=float32)

In [14]:
import tf2onnx
import onnxruntime as rt

In [15]:
spec = (tf.TensorSpec((None, 28, 28, 1), tf.float32, name="input"),)
output_path = './Data/conv_flatten_softmax_model.onnx'

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

2022-01-28 09:42:22.428257: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-01-28 09:42:22.428486: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-01-28 09:42:22.434043: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.024ms.
  function_optimizer: function_optimizer did nothing. time = 0.003ms.

2022-01-28 09:42:22.574727: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-01-28 09:42:22.575019: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-01-28 09:42:22.586257: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 25 nodes (-8), 32 edges (

In [17]:
# Run the ONNX model
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {"input": [a_val_img]})

In [19]:
onnx_pred[0]

array([[2.7691520e-19, 2.1103689e-18, 6.9270651e-15, 4.4732587e-08,
        1.5199857e-23, 2.5407996e-17, 8.9123905e-38, 1.0000000e+00,
        1.3648736e-15, 1.2447636e-13]], dtype=float32)

In [20]:
model.predict(np.array([a_val_img]))

array([[2.7691417e-19, 2.1103689e-18, 6.9270913e-15, 4.4732502e-08,
        1.5199857e-23, 2.5408093e-17, 8.9121865e-38, 1.0000000e+00,
        1.3648736e-15, 1.2447636e-13]], dtype=float32)